In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from pymongo import MongoClient
from dotenv import load_dotenv

In [2]:
# Nama database dan collection
db_name = 'youtube_rewind_indonesia'
collection_label = 'labeling_2021'
collection_name = 'full_labeling_2021'

# Memuat value dari file .env
load_dotenv()

mongodb_url = os.getenv('URL_SANDY')
local_url = os.getenv('URL_LOCAL')

#### Mengambil data dari MongoDB ke dalam dataframe

In [3]:
# Membuat koneksi ke MongoDB
client = MongoClient(local_url)
db = client[db_name]  # Ganti dengan nama database Anda
labeling_2018_collection = db[collection_label]
full_labeling_2018_collection = db[collection_name]

# Mengambil data dari koleksi MongoDB
labeled_data = list(labeling_2018_collection.find())
unlabeled_data = list(full_labeling_2018_collection.find())

# Konversi data menjadi dataframe
df_labeled = pd.DataFrame(labeled_data)
df_unlabeled = pd.DataFrame(unlabeled_data)

# Memisahkan fitur dan label dari data yang berlabel
X_labeled = df_labeled['textOriginal']
y_labeled = df_labeled['label']

# Menggunakan TfidfVectorizer untuk mengubah teks menjadi fitur
vectorizer = TfidfVectorizer()
X_labeled_tfidf = vectorizer.fit_transform(X_labeled)

# Melatih model menggunakan Naive Bayes
model = MultinomialNB()
model.fit(X_labeled_tfidf, y_labeled)

# Memproses data yang tidak berlabel
X_unlabeled = df_unlabeled['textOriginal']
X_unlabeled_tfidf = vectorizer.transform(X_unlabeled)

# Memprediksi label untuk data yang tidak berlabel
y_unlabeled_pred = model.predict(X_unlabeled_tfidf)

# Menambahkan prediksi ke dalam dataframe data tidak berlabel
df_unlabeled['predicted_label'] = y_unlabeled_pred

# Mengupdate koleksi MongoDB dengan prediksi label
for index, row in df_unlabeled.iterrows():
    full_labeling_2018_collection.update_one(
        {"_id": row["_id"]},
        {"$set": {"predicted_label": row["predicted_label"]}}
    )

print("Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.")

Prediksi label untuk data tidak berlabel telah selesai dan diupdate dalam koleksi MongoDB.


In [4]:
df_positif = df_unlabeled[df_unlabeled['predicted_label'] == 'positif']
df_negatif = df_unlabeled[df_unlabeled['predicted_label'] == 'negatif']
df_netral = df_unlabeled[df_unlabeled['predicted_label'] == 'netral']

In [5]:
df_positif

,_id,textOriginal,predicted_label
0,6651a129ba72d8bf2eb58341,semua cerita pesan sangat mudah cerna god job ...,positif
2,6651a129ba72d8bf2eb58343,keren banget banget banget plis audio kualitas...,positif
3,6651a129ba72d8bf2eb58344,the best ni,positif
4,6651a129ba72d8bf2eb58345,keren keren sangat keren,positif
6,6651a129ba72d8bf2eb58347,chandra top,positif
...,...,...,...
9992,6651a129ba72d8bf2eb5aa49,dovijovi,positif
9993,6651a129ba72d8bf2eb5aa4a,baik sedih pas liat anak ibu ke rumah nyata an...,positif
9994,6651a129ba72d8bf2eb5aa4b,style konsep video inspirasi wonderland indone...,positif
9995,6651a129ba72d8bf2eb5aa4c,awal selesai gk henti linang air mata sumpah t...,positif


In [6]:
df_negatif

,_id,textOriginal,predicted_label
10,6651a129ba72d8bf2eb5834b,jelek ah sponsor makan tu sponsor kwkwkwkw,negatif
172,6651a129ba72d8bf2eb583ed,tiktok rewind mah awokwk,negatif
239,6651a129ba72d8bf2eb58430,yt rewind apa rewindyang trend malah ajakada y...,negatif
249,6651a129ba72d8bf2eb5843a,mah tik tok rewind dah,negatif
280,6651a129ba72d8bf2eb58459,semua tik tok,negatif
...,...,...,...
9610,6651a129ba72d8bf2eb5a8cb,tiktok rewind,negatif
9689,6651a129ba72d8bf2eb5a91a,bocah tiktok serang,negatif
9778,6651a129ba72d8bf2eb5a973,rewind tik tok ker indonesia,negatif
9806,6651a129ba72d8bf2eb5a98f,bukan youtube mah tik tok,negatif


In [7]:
df_netral

,_id,textOriginal,predicted_label
1,6651a129ba72d8bf2eb58342,salam binjai wkwk,netral
5,6651a129ba72d8bf2eb58346,kecewa daoa ngak masuk,netral
7,6651a129ba72d8bf2eb58348,maf gk berkatakata lagisebuah karya penuh tota...,netral
9,6651a129ba72d8bf2eb5834a,rehan bogor mana indonesia mana,netral
13,6651a129ba72d8bf2eb5834e,rewind indonesia jelek pernah ada iklan kocak ...,netral
...,...,...,...
9987,6651a129ba72d8bf2eb5aa44,sini brayan furan sama miaug,netral
9989,6651a129ba72d8bf2eb5aa46,wkwkw sensor,netral
9996,6651a129ba72d8bf2eb5aa4d,bryan furan anjay,netral
9998,6651a129ba72d8bf2eb5aa4f,halilingis mana si number one wkwowkwk,netral


#### Accuracy Test (soon)

In [8]:
# Jika data tidak berlabel memiliki label asli untuk keperluan evaluasi
if 'label' in df_unlabeled.columns:
    print("\nEvaluasi kinerja model:")
    print("Accuracy Score:", accuracy_score(df_unlabeled['label'], df_unlabeled['predicted_label']))
    print("Classification Report:\n", classification_report(df_unlabeled['label'], df_unlabeled['predicted_label']))
else:
    print("Data tidak berlabel tidak memiliki label asli untuk evaluasi.")

Data tidak berlabel tidak memiliki label asli untuk evaluasi.
